# Đồ án cuối kỳ : Tối ưu hóa Radix Sort trên GPU

Thông tin sinh viên :

* Hoàng Minh Thanh (18424062)
* Nguyễn Mạnh Tấn (18424060)

### Kết quả tốt nhất :
BlockSize (256), kBit = 8 (Tốn thời gian 250% so với thrust, hay nhanh gần 40% so với thrust)
```
Radix Sort by device
  Time: 164.896 ms
Radix Sort with thrust
  Time: 64.943 ms
```

Respo Github : https://github.com/hmthanh/radix-sort-cuda

* Cập nhật (11\/ 10\/ 2020) : Kết quả tốt nhất :
blockSize = 512; kBit = 8

```
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 512

Radix Sort by host
Time: 1157.686 ms

Baseline Radix Sort (highlight)
Time: 6223.896 ms
CORRECT :)

Radix Sort by device
Time: 107.973 ms
CORRECT :)

Radix Sort with thrust
Time: 61.450 ms
CORRECT :)
```

# A. Cài đặt plugin

Cài đặt plugin chạy GPU trên Google Colab

In [1]:
%%bash
nvcc --version
rm -rf /content/*
pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-brozph9x
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=fd83d1f4a7db61870414113595a8431264e55fd1f5c08efee4de23ccdb8d7a96
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdx6rjg1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-brozph9x


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# B. Mã nguồn chương trình Radix Sort 

## 1. Radix sort sử dụng Thrush

In [158]:
%%cu
#include <stdio.h>
#include <stdint.h>
#include <thrust/device_vector.h>
#include <thrust/copy.h>
#include <thrust/sort.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
	// In each loop, sort elements according to the current digit from src to dst 
	// (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
    	// TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

    	// TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

    	// TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
    	
    	// Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int blockSize)
{
    // TODO
    thrust::device_vector<uint32_t> dv_out(in, in + n);
    thrust::sort(dv_out.begin(), dv_out.end());
    thrust::copy(dv_out.begin(), dv_out.end(), out);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
    	printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
    	printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = 10; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 10
163 151 162 85 83 190 241 252 249 121 

Radix Sort by host
Time: 0.012 ms
83 85 121 151 162 163 190 241 249 252 

Radix Sort by device
Time: 0.329 ms
CORRECT :)



## 2. Radix sort tuần tự

In [160]:
%%cu
/*
Baseline 1 : Thuật toán Radix Sort tuần tự
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Base 1
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int blockSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    //printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    //printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 4362.573 ms

Radix Sort by device
Time: 4503.639 ms
CORRECT :)



## 2. Baseline 1 : Thực Song song 2 bước hist và scan

### 2.1 Song song tính histogram

In [161]:
%%cu
/*
Baseline 2.1 : Song song 2 bước hist
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // Each block compute its local hist using atomic on SMEM
    extern __shared__ int s_bin[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int delta = (nBins - 1) / blockDim.x + 1;
    for (int i = 0; i < delta; i++){
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            s_bin[id] = 0;
        }
    }
    __syncthreads();

    if (i < n){
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&s_bin[bin], 1);
    }
    __syncthreads();

    // Each block adds its local hist to global hist using atomic on GMEM
    for (int i = 0; i < delta; i++)
    {
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            atomicAdd(&hist[id], s_bin[id]);
        }
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }

    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (threadIdx.x == 0 && blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size 
    size_t smemHistBytes = nBins * sizeof(int);

    int * d_hist;
    CHECK(cudaMalloc(&d_hist, nBins * sizeof(int)));

    uint32_t *d_src;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice));

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits){
        // // TODO: Compute histogram
        // memset(hist, 0, nBins * sizeof(int));
        CHECK(cudaMemset(d_hist, 0, nBins * sizeof(int)));
        // for (int i = 0; i < n; i++)
        // {
        //     int bin = (src[i] >> bit) & (nBins - 1);
        //     hist[bin]++;
        // }
        // Compute histogram
        computeHistogram<<<gridSize, blockSize, smemHistBytes>>>(d_src, n, d_hist, nBins, bit);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(hist, d_hist, nBins * sizeof(int), cudaMemcpyDeviceToHost));

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    //printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    //printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 4378.262 ms

Radix Sort by device
Time: 3043.682 ms
CORRECT :)



### 2.2 Thực hiện song song scan

In [162]:
%%cu
/*
Baseline 2.2 : Song song 2 bước hist và scan
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // Each block compute its local hist using atomic on SMEM
    extern __shared__ int s_bin[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int delta = (nBins - 1) / blockDim.x + 1;
    for (int i = 0; i < delta; i++){
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            s_bin[id] = 0;
        }
    }
    __syncthreads();

    if (i < n){
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&s_bin[bin], 1);
    }
    __syncthreads();

    // Each block adds its local hist to global hist using atomic on GMEM
    for (int i = 0; i < delta; i++)
    {
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            atomicAdd(&hist[id], s_bin[id]);
        }
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }

    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (threadIdx.x == 0 && blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

// Sum scan result of its previous block
__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n && blockIdx.x > 0){
        in[i] += blkSums[blockIdx.x - 1];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits
    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size 
    size_t smemHistBytes = nBins * sizeof(int);
    size_t smemScanBytes = blockSize.x * sizeof(int);
    
    int * d_hist, *d_histScan, *d_blkSums;
    CHECK(cudaMalloc(&d_hist, nBins * sizeof(int)));
    CHECK(cudaMalloc(&d_histScan, nBins * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridSize.x * sizeof(int)));

    uint32_t *d_src;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice));

    int * blkSums;
    blkSums = (int*)malloc(gridSize.x * sizeof(int));

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        CHECK(cudaMemset(d_hist, 0, nBins * sizeof(int)));
        
        // Compute histogram
        computeHistogram<<<gridSize, blockSize, smemHistBytes>>>(d_src, n, d_hist, nBins, bit);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(hist, d_hist, nBins * sizeof(int), cudaMemcpyDeviceToHost));

        // Scan exclusive only its block
        scanExclusiveBlk<<<gridSize, blockSize, smemScanBytes>>>(d_hist, nBins, d_histScan, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridSize.x * sizeof(int), cudaMemcpyDeviceToHost));

        // Sum scan result of its previous block
        for (int i = 1; i < gridSize.x; i++){
            blkSums[i] += blkSums[i-1];
        }
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridSize, blockSize>>>(d_histScan, nBins, d_blkSums);
        cudaDeviceSynchronize();

        CHECK(cudaMemcpy(histScan, d_histScan, nBins * sizeof(int), cudaMemcpyDeviceToHost));

        // Scatter
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_hist));
    CHECK(cudaFree(d_blkSums));
    CHECK(cudaFree(d_histScan));
    memcpy(out, src, n * sizeof(uint32_t));
    
    free(blkSums);
    free(hist);
    free(histScan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1155.662 ms

Radix Sort by device
Time: 2847.243 ms
CORRECT :)



## 3. Thực hiện radix sort với k = 1

In [164]:
%%cu
/*
Baseline 3 : Thực hiện radix sort với k = 1
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // Each block compute its local hist using atomic on SMEM
    extern __shared__ int s_bin[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int delta = (nBins - 1) / blockDim.x + 1;
    for (int i = 0; i < delta; i++){
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            s_bin[id] = 0;
        }
    }
    __syncthreads();

    if (i < n){
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&s_bin[bin], 1);
    }
    __syncthreads();

    // Each block adds its local hist to global hist using atomic on GMEM
    for (int i = 0; i < delta; i++)
    {
        int id = threadIdx.x + i * blockDim.x;
        if (id < nBins){
            atomicAdd(&hist[id], s_bin[id]);
        }
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }

    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (threadIdx.x == 0 && blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

// Sum scan result of its previous block
__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n && blockIdx.x > 0){
        in[i] += blkSums[blockIdx.x - 1];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits
    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size 
    size_t smemHistBytes = nBins * sizeof(int);
    size_t smemScanBytes = blockSize.x * sizeof(int);
    
    int * d_hist, *d_histScan, *d_blkSums;
    CHECK(cudaMalloc(&d_hist, nBins * sizeof(int)));
    CHECK(cudaMalloc(&d_histScan, nBins * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridSize.x * sizeof(int)));

    uint32_t *d_src;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice));

    int * blkSums;
    blkSums = (int*)malloc(gridSize.x * sizeof(int));

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        CHECK(cudaMemset(d_hist, 0, nBins * sizeof(int)));
        
        // Compute histogram
        computeHistogram<<<gridSize, blockSize, smemHistBytes>>>(d_src, n, d_hist, nBins, bit);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(hist, d_hist, nBins * sizeof(int), cudaMemcpyDeviceToHost));

        // Scan exclusive only its block
        scanExclusiveBlk<<<gridSize, blockSize, smemScanBytes>>>(d_hist, nBins, d_histScan, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridSize.x * sizeof(int), cudaMemcpyDeviceToHost));

        // Sum scan result of its previous block
        for (int i = 1; i < gridSize.x; i++){
            blkSums[i] += blkSums[i-1];
        }
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridSize, blockSize>>>(d_histScan, nBins, d_blkSums);
        cudaDeviceSynchronize();

        CHECK(cudaMemcpy(histScan, d_histScan, nBins * sizeof(int), cudaMemcpyDeviceToHost));

        // Scatter
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_hist));
    CHECK(cudaFree(d_blkSums));
    CHECK(cudaFree(d_histScan));
    memcpy(out, src, n * sizeof(uint32_t));
    
    free(blkSums);
    free(hist);
    free(histScan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1153.028 ms

Radix Sort by device
Time: 2915.646 ms
CORRECT :)



## 4. Thuật toán Radix Sort tuần tự theo hướng dẫn đồ án

### Baseline 1 : 
Cài đặt mảng lưu bin như file hướng dẫn. (hàng là các block, cột là các bin)

In [165]:
%%cu
/*
Baseline 4 : Thuật toán Radix Sort tuần tự theo hướng dẫn đồ án
Cài đặt mảng lưu bin như file hướng dẫn. (hàng là các block, cột là các bin)
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    // Free memories
    free(hist);
    free(histScan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1155.415 ms

Radix Sort by device
Time: 6217.125 ms
CORRECT :)



### Baseline 2 : Song song hist và scan

In [167]:
%%cu
/*
Baseline 4.2 : Song song 2 bước tính hist và scan*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// #########################################################
// Baseline
void sortBaseline(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        // compute historgram
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        // compute scan
        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        // scatter
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    free(hist);
    free(histScan);
}

// #########################################################
// Radix sort by device
// #########################################################
// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // TODO
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
    {
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&hist[bin * gridDim.x + blockIdx.x], 1);
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{   
    // TODO
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }
    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n && blockIdx.x > 0)
        in[i] += blkSums[blockIdx.x - 1];
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(bklSize);
    dim3 gridHistSize((n - 1) / blockSize.x + 1);
    dim3 gridScanSize((nBins * gridHistSize.x - 1) / blockSize.x + 1);
    
    int * scan = (int * )malloc(nBins * gridHistSize.x * sizeof(int));
    int * blkSums = (int *)malloc(gridScanSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    uint32_t * d_src;
    int *d_hist, *d_scan, *d_blkSums;

    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMalloc(&d_hist, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_scan, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridScanSize.x * sizeof(int)));

    size_t smemHistBytes = nBins * sizeof(int); 
    size_t smemScanBytes = blockSize.x * sizeof(int);
    
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // compute historgram
        CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice));
        CHECK(cudaMemset(d_hist, 0, nBins * gridHistSize.x * sizeof(int)));
        computeHistogram<<<gridHistSize, blockSize, smemHistBytes>>>(d_src, n, d_hist, nBins, bit);
        cudaDeviceSynchronize();

        // compute scan
        scanExclusiveBlk<<<gridScanSize, blockSize, smemScanBytes>>>(d_hist, nBins * gridHistSize.x, d_scan, d_blkSums);
        cudaDeviceSynchronize();
        
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridScanSize.x * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 1; i < gridScanSize.x; i++){
            blkSums[i] += blkSums[i - 1];
        }
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridScanSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridScanSize, blockSize>>>(d_scan, nBins * gridHistSize.x, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(scan, d_scan, nBins * gridHistSize.x * sizeof(int), cudaMemcpyDeviceToHost)); 
        
        // Scatter
        for (int i = 0; i < n ; i++)
        {
            int bin = i / blockSize.x + ((src[i] >> bit) & (nBins - 1)) * gridHistSize.x;
            dst[scan[bin]] = src[i];
            scan[bin]++;
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 

    }
    // Copy result to "out"
    memcpy(out, src, n * sizeof(uint32_t));

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_hist));
    CHECK(cudaFree(d_scan));
    CHECK(cudaFree(d_blkSums));
    
    free(blkSums);
    free(scan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1, int type=0)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else if (type == 1){ // Baseline
        printf("\nBaseline Radix Sort (highlight)\n");
        sortBaseline(in, n, out, blockSize);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out_baseline = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);

    // SORT BY BASELINE
    sort(in, n, out_baseline, true, blockSize, 1);
    checkCorrectness(out_baseline, correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize, 2);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1139.955 ms

Baseline Radix Sort (highlight)
Time: 6190.423 ms
CORRECT :)

Radix Sort by device
Time: 4623.783 ms
CORRECT :)



### Baseline 3 : Cài đặt preScatter và scatter song song

In [169]:
%%cu
/*
Baseline 4.3 : Cài đặt preScatter và scatter song song
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// #########################################################
// Baseline
void sortBaseline(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(bklSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        // compute historgram
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        // compute scan
        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        // scatter
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    free(hist);
    free(histScan);
}

// #########################################################
// Radix sort by device
// #########################################################
// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // TODO
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
    {
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&hist[bin * gridDim.x + blockIdx.x], 1);
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{   
    // TODO
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }
    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n && blockIdx.x > 0)
        in[i] += blkSums[blockIdx.x - 1];
}

__global__ void preScatter(uint32_t * in, int n, int nBits, int bit, int nBins, int * out)
{
    extern __shared__ int s_data[];
    int * s_in = s_data;
    int * s_hist = (int *)&s_in[blockDim.x];
    int * dst = (int *)&s_hist[blockDim.x];
    int * dst_ori = (int *)&dst[blockDim.x];
    int * startIndex = (int *)&dst_ori[blockDim.x];
    int * hist = (int *)&startIndex[blockDim.x];
    int * scan = (int *)&hist[blockDim.x];

    int id = blockIdx.x * blockDim.x + threadIdx.x;
    if (id < n) {
        s_in[threadIdx.x] = in[id];
        s_hist[threadIdx.x] = (s_in[threadIdx.x] >> bit) & (nBins - 1); // get bit
    }
    else {
        s_hist[threadIdx.x] = nBins - 1;
    }
    __syncthreads();

    // Step 1 : sort radix with k = 1
    for (int b = 0; b < nBits; b++)
    {
        // compute hist and scan
        hist[threadIdx.x] = (s_hist[threadIdx.x] >> b) & 1;
        __syncthreads();
        if (threadIdx.x == 0)
            scan[0] = 0;
        else
            scan[threadIdx.x] = hist[threadIdx.x - 1];
        __syncthreads();

        // scan
        for (int stride = 1; stride < blockDim.x; stride *= 2)
        {
            int val = 0;
            if (threadIdx.x >= stride){
                val = scan[threadIdx.x - stride];
            }
            __syncthreads();

            scan[threadIdx.x] += val;
            __syncthreads();
        }
        __syncthreads();

        int nZeros = blockDim.x - scan[blockDim.x - 1] - hist[blockDim.x - 1];
        int rank = 0;
        if (hist[threadIdx.x] == 0){
            rank = threadIdx.x - scan[threadIdx.x];
        }
        else{
            rank = nZeros + scan[threadIdx.x];
        }
        dst[rank] = s_hist[threadIdx.x];
        dst_ori[rank] = s_in[threadIdx.x];
        __syncthreads();

        // copy
        s_hist[threadIdx.x] = dst[threadIdx.x];
        s_in[threadIdx.x] = dst_ori[threadIdx.x];
    }
    __syncthreads();

    // Step 2
    if (threadIdx.x == 0) {
        startIndex[s_hist[0]] = 0;
    }
    else{
        if (s_hist[threadIdx.x] != s_hist[threadIdx.x - 1]){
            startIndex[s_hist[threadIdx.x]] = threadIdx.x;
        }
    }
    __syncthreads();

    // Step 3
    if (id < n) {
        out[id] = threadIdx.x - startIndex[s_hist[threadIdx.x]];
        in[id] = s_in[threadIdx.x];
    }
}

__global__ void scatter(uint32_t * in, int * preRank, int bit, 
                        int *histScan, int n, int nBins, uint32_t *out)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
    {
        int bin = ((in[i] >> bit) & (nBins - 1));
        int scan = histScan[bin * gridDim.x + blockIdx.x];
        int rank = scan + preRank[i];
        out[rank] = in[i];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int bklSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(bklSize);
    dim3 gridHistSize((n - 1) / blockSize.x + 1);
    dim3 gridScanSize((nBins * gridHistSize.x - 1) / blockSize.x + 1);
    dim3 gridScatterSize((n - 1) / blockSize.x + 1);
    
    int * scan = (int * )malloc(nBins * gridHistSize.x * sizeof(int));
    int * blkSums = (int *)malloc(gridScanSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));

    int *d_hist, *d_scan, *d_blkSums;
    CHECK(cudaMalloc(&d_hist, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_scan, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridScanSize.x * sizeof(int)));

    uint32_t * d_src, *d_dst;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice)); // copy to device
    CHECK(cudaMalloc(&d_dst, n * sizeof(uint32_t)));

    size_t smemBytes = blockSize.x * sizeof(int);
    size_t smemScatterBytes = blockSize.x * 7 * sizeof(int);

    int * d_preRank; 
    CHECK(cudaMalloc(&d_preRank, n * sizeof(int)));
    
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // Compute "hist" of the current digit
        CHECK(cudaMemset(d_scan, 0, nBins * gridHistSize.x * sizeof(int)));
        computeHistogram<<<gridHistSize, blockSize, smemBytes>>>(d_src, n, d_scan, nBins, bit);
        cudaDeviceSynchronize();
        

        // Scan
        scanExclusiveBlk<<<gridScanSize, blockSize, smemBytes>>>(d_scan, nBins * gridHistSize.x, d_scan, d_blkSums);
        cudaDeviceSynchronize();
        
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridScanSize.x * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 1; i < gridScanSize.x; i++){
            blkSums[i] += blkSums[i - 1];
        }
        
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridScanSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridScanSize, blockSize>>>(d_scan, nBins * gridHistSize.x, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(scan, d_scan, sizeof(int)* nBins * gridHistSize.x, cudaMemcpyDeviceToHost));

        // Scatter
        preScatter<<<gridScatterSize, blockSize, smemScatterBytes>>>(d_src, n, nBits, bit, nBins, d_preRank);
        cudaDeviceSynchronize();
        
        scatter<<<gridScatterSize, blockSize>>>(d_src, d_preRank, bit, d_scan, n, nBins, d_dst);
        cudaDeviceSynchronize();
        
        // Swap "src" and "dst"
        uint32_t * temp = d_src;
        d_src = d_dst;
        d_dst = temp;
    }
    // Copy "d_src" to "out"
    CHECK(cudaMemcpy(out, d_src, n * sizeof(uint32_t), cudaMemcpyDeviceToHost));

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_hist));
    CHECK(cudaFree(d_scan));
    CHECK(cudaFree(d_blkSums));
    
    free(blkSums);
    free(scan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1, int type=0)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else if (type == 1){ // Baseline
        printf("\nBaseline Radix Sort (highlight)\n");
        sortBaseline(in, n, out, blockSize);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out_baseline = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);

    // SORT BY BASELINE
    sort(in, n, out_baseline, true, blockSize, 1);
    checkCorrectness(out_baseline, correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize, 2);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1135.346 ms

Baseline Radix Sort (highlight)
Time: 6158.176 ms
CORRECT :)

Radix Sort by device
Time: 352.382 ms
CORRECT :)



### Baseline 4 Cài Scatter song song

In [170]:
%%cu
/*
Baseline 4.4 : Cài đặt scatter song song
*/
#include <stdio.h>
#include <stdint.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// #########################################################
// Baseline
void sortBaseline(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        // compute historgram
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        // compute scan
        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        // scatter
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    free(hist);
    free(histScan);
}

// #########################################################
// Radix sort by device
// #########################################################
// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // TODO
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
    {
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&hist[bin * gridDim.x + blockIdx.x], 1);
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{   
    // TODO
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }
    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n && blockIdx.x > 0)
        in[i] += blkSums[blockIdx.x - 1];
}

__global__ void scatter(uint32_t * in, int n, int nBits, int bit, int nBins, int *histScan, uint32_t * out)
{
    extern __shared__ int s_data[];
    int * s_in = s_data;
    int * s_hist = (int *)&s_in[blockDim.x];
    int * dst = (int *)&s_hist[blockDim.x];
    int * dst_ori = (int *)&dst[blockDim.x];
    int * startIndex = (int *)&dst_ori[blockDim.x];
    int * hist = (int *)&startIndex[blockDim.x];
    int * scan = (int *)&hist[blockDim.x];

    int id = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (id < n){
        s_in[threadIdx.x] = in[id];
        s_hist[threadIdx.x] = (s_in[threadIdx.x] >> bit) & (nBins - 1); // get bit
    }
    else {
        s_hist[threadIdx.x] = nBins - 1;
    }
    // Step 1 : sort radix with k = 1
    for (int b = 0; b < nBits; b++){
        // compute hist
        hist[threadIdx.x] = (s_hist[threadIdx.x] >> b) & 1;
        __syncthreads();

        // scan
        if (threadIdx.x == 0){
            scan[0] = 0;
        }
        else {
            scan[threadIdx.x] = hist[threadIdx.x - 1];
        }
        __syncthreads();

        for (int stride = 1; stride < blockDim.x; stride *= 2) {
            int val = 0;
            if (threadIdx.x >= stride){
                val = scan[threadIdx.x - stride];
            }
            __syncthreads();

            scan[threadIdx.x] += val;
            __syncthreads();
        }
        __syncthreads();

        // scatter
        int nZeros = blockDim.x - scan[blockDim.x - 1] - hist[blockDim.x - 1];
        int rank = 0;
        if (hist[threadIdx.x] == 0){
            rank = threadIdx.x - scan[threadIdx.x];
        }
        else{
            rank = nZeros + scan[threadIdx.x];
        }
        dst[rank] = s_hist[threadIdx.x];
        dst_ori[rank] = s_in[threadIdx.x];
        __syncthreads();

        // copy or swap
        s_hist[threadIdx.x] = dst[threadIdx.x];
        s_in[threadIdx.x] = dst_ori[threadIdx.x];
    }
    __syncthreads();

    // Step 2
    if (threadIdx.x == 0){
        startIndex[s_hist[0]] = 0;
    }
    else
    {
        if (s_hist[threadIdx.x] != s_hist[threadIdx.x - 1]){
            startIndex[s_hist[threadIdx.x]] = threadIdx.x;
        }
    }
    __syncthreads();

    // Step 3
    if (id < n)
    {
        int preRank = threadIdx.x - startIndex[s_hist[threadIdx.x]];
        int bin = ((s_in[threadIdx.x] >> bit) & (nBins - 1));
        int scan = histScan[bin * gridDim.x + blockIdx.x];
        int rank = scan + preRank;
        out[rank] = s_in[threadIdx.x];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize);
    dim3 gridHistSize((n - 1) / blockSize.x + 1);
    dim3 gridScanSize((nBins * gridHistSize.x - 1) / blockSize.x + 1);
    dim3 gridScatterSize((n - 1) / blockSize.x + 1);
    
    int * scan = (int * )malloc(nBins * gridHistSize.x * sizeof(int));
    int * blkSums = (int *)malloc(gridScanSize.x * sizeof(int));

    int *d_scan, *d_blkSums;
    CHECK(cudaMalloc(&d_scan, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridScanSize.x * sizeof(int)));

    uint32_t * d_src, *d_dst;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, in, n * sizeof(uint32_t), cudaMemcpyHostToDevice)); // copy to device
    CHECK(cudaMalloc(&d_dst, n * sizeof(uint32_t)));

    size_t smemBytes = blockSize.x * sizeof(int);
    size_t smemScatterBytes = blockSize.x * 7 * sizeof(int);
    
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // Compute "hist" of the current digit
        CHECK(cudaMemset(d_scan, 0, nBins * gridHistSize.x * sizeof(int)));
        computeHistogram<<<gridHistSize, blockSize, smemBytes>>>(d_src, n, d_scan, nBins, bit);
        cudaDeviceSynchronize();
        

        // Scan
        scanExclusiveBlk<<<gridScanSize, blockSize, smemBytes>>>(d_scan, nBins * gridHistSize.x, d_scan, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridScanSize.x * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 1; i < gridScanSize.x; i++){
            blkSums[i] += blkSums[i - 1];
        }
        
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridScanSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridScanSize, blockSize>>>(d_scan, nBins * gridHistSize.x, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(scan, d_scan, sizeof(int)* nBins * gridHistSize.x, cudaMemcpyDeviceToHost));

        // Scatter
        scatter<<<gridScatterSize, blockSize, smemScatterBytes>>>(d_src, n, nBits, bit, nBins, d_scan, d_dst);
        cudaDeviceSynchronize();
        
        // Swap "src" and "dst"
        uint32_t * temp = d_src;
        d_src = d_dst;
        d_dst = temp;
    }
    // Copy "d_src" to "out"
    CHECK(cudaMemcpy(out, d_src, n * sizeof(uint32_t), cudaMemcpyDeviceToHost));

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_scan));
    CHECK(cudaFree(d_blkSums));
    
    free(blkSums);
    free(scan);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1, int type=0)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else if (type == 1){ // Baseline
        printf("\nBaseline Radix Sort (highlight)\n");
        sortBaseline(in, n, out, blockSize);
    }
    else // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out_baseline = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default 
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);

    // SORT BY BASELINE
    sort(in, n, out_baseline, true, blockSize, 1);
    checkCorrectness(out_baseline, correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize, 2);
    checkCorrectness(out, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1126.321 ms

Baseline Radix Sort (highlight)
Time: 6222.790 ms
CORRECT :)

Radix Sort by device
Time: 292.879 ms
CORRECT :)



### Radix sort final

 Sort với bit = 8

In [171]:
%%cu
/*
Radix sort final
*/
#include <stdio.h>
#include <stdint.h>
#include <thrust/device_vector.h>
#include <thrust/copy.h>
#include <thrust/sort.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// #########################################################
// Baseline
void sortBaseline(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        // compute historgram
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        // compute scan
        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        // scatter
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    free(hist);
    free(histScan);
}

// #########################################################
// Radix sort by device
// #########################################################
// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // TODO
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
    {
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&hist[bin * gridDim.x + blockIdx.x], 1);
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{   
    // TODO
    extern __shared__ int s_data[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[threadIdx.x] = in[i - 1];
    }
    else{
        s_data[threadIdx.x] = 0;
    }
    __syncthreads();
    
    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x >= stride){
            val = s_data[threadIdx.x - stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }
    if (i < n){
        out[i] = s_data[threadIdx.x];
    }
    if (blkSums != NULL){
        blkSums[blockIdx.x] = s_data[blockDim.x - 1];
    }
}

__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n && blockIdx.x > 0)
        in[i] += blkSums[blockIdx.x - 1];
}

__global__ void scatter(uint32_t * in, int n, int nBits, int bit, int nBins, int *histScan, uint32_t * out)
{
    extern __shared__ int s_data[];
    int * s_in = s_data;
    int * s_hist = (int *)&s_in[blockDim.x];
    int * dst = (int *)&s_hist[blockDim.x];
    int * dst_ori = (int *)&dst[blockDim.x];
    int * startIndex = (int *)&dst_ori[blockDim.x];
    int * hist = (int *)&startIndex[blockDim.x];
    int * scan = (int *)&hist[blockDim.x];

    int id = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (id < n){
        s_in[threadIdx.x] = in[id];
        s_hist[threadIdx.x] = (s_in[threadIdx.x] >> bit) & (nBins - 1); // get bit
    }
    else {
        s_hist[threadIdx.x] = nBins - 1;
    }
    // Step 1 : sort radix with k = 1
    for (int b = 0; b < nBits; b++){
        // compute hist
        hist[threadIdx.x] = (s_hist[threadIdx.x] >> b) & 1;
        __syncthreads();

        // scan
        if (threadIdx.x == 0){
            scan[0] = 0;
        }
        else {
            scan[threadIdx.x] = hist[threadIdx.x - 1];
        }
        __syncthreads();

        for (int stride = 1; stride < blockDim.x; stride *= 2) {
            int val = 0;
            if (threadIdx.x >= stride){
                val = scan[threadIdx.x - stride];
            }
            __syncthreads();

            scan[threadIdx.x] += val;
            __syncthreads();
        }
        __syncthreads();

        // scatter
        int nZeros = blockDim.x - scan[blockDim.x - 1] - hist[blockDim.x - 1];
        int rank = 0;
        if (hist[threadIdx.x] == 0){
            rank = threadIdx.x - scan[threadIdx.x];
        }
        else{
            rank = nZeros + scan[threadIdx.x];
        }
        dst[rank] = s_hist[threadIdx.x];
        dst_ori[rank] = s_in[threadIdx.x];
        __syncthreads();

        // copy or swap
        s_hist[threadIdx.x] = dst[threadIdx.x];
        s_in[threadIdx.x] = dst_ori[threadIdx.x];
    }
    __syncthreads();

    // Step 2
    if (threadIdx.x == 0){
        startIndex[s_hist[0]] = 0;
    }
    else
    {
        if (s_hist[threadIdx.x] != s_hist[threadIdx.x - 1]){
            startIndex[s_hist[threadIdx.x]] = threadIdx.x;
        }
    }
    __syncthreads();

    // Step 3
    if (id < n)
    {
        int preRank = threadIdx.x - startIndex[s_hist[threadIdx.x]];
        int bin = ((s_in[threadIdx.x] >> bit) & (nBins - 1));
        int scan = histScan[bin * gridDim.x + blockIdx.x];
        int rank = scan + preRank;
        out[rank] = s_in[threadIdx.x];
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 8; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize);
    dim3 gridHistSize((n - 1) / blockSize.x + 1);
    dim3 gridScanSize((nBins * gridHistSize.x - 1) / blockSize.x + 1);
    dim3 gridScatterSize((n - 1) / blockSize.x + 1);
    
    int * blkSums = (int *)malloc(gridScanSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));

    int *d_hist, *d_scan, *d_blkSums;
    CHECK(cudaMalloc(&d_hist, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_scan, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridScanSize.x * sizeof(int)));

    uint32_t * d_src, *d_dst;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, src, n * sizeof(uint32_t), cudaMemcpyHostToDevice)); // copy to device
    CHECK(cudaMalloc(&d_dst, n * sizeof(uint32_t)));

    size_t smemBytes = blockSize.x * sizeof(int);
    size_t smemScatterBytes = blockSize.x * 7 * sizeof(int);
    
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // Compute "hist" of the current digit
        CHECK(cudaMemset(d_scan, 0, nBins * gridHistSize.x * sizeof(int)));
        computeHistogram<<<gridHistSize, blockSize, smemBytes>>>(d_src, n, d_scan, nBins, bit);
        cudaDeviceSynchronize();
        
        // Scan
        scanExclusiveBlk<<<gridScanSize, blockSize, smemBytes>>>(d_scan, nBins * gridHistSize.x, d_scan, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridScanSize.x * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 1; i < gridScanSize.x; i++){
            blkSums[i] += blkSums[i - 1];
        }
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridScanSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridScanSize, blockSize>>>(d_scan, nBins * gridHistSize.x, d_blkSums);
        cudaDeviceSynchronize();

        // Scatter
        scatter<<<gridScatterSize, blockSize, smemScatterBytes>>>(d_src, n, nBits, bit, nBins, d_scan, d_dst);
        cudaDeviceSynchronize();
        
        // Swap "src" and "dst"
        uint32_t * temp = d_src;
        d_src = d_dst;
        d_dst = temp;
    }
    // Copy "d_src" to "out"
    CHECK(cudaMemcpy(out, d_src, n * sizeof(uint32_t), cudaMemcpyDeviceToHost));

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_hist));
    CHECK(cudaFree(d_scan));
    CHECK(cudaFree(d_blkSums));
    
    free(blkSums);
}

// #########################################################
// Radix sort by thrust
// #########################################################
void sortWithThrust(const uint32_t * in, int n, uint32_t * out)
{
    thrust::device_vector<uint32_t> dv_out(in, in + n);
    thrust::sort(dv_out.begin(), dv_out.end());
    thrust::copy(dv_out.begin(), dv_out.end(), out);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1, int type=0)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else if (type == 1){ // Baseline
        printf("\nBaseline Radix Sort (highlight)\n");
        sortBaseline(in, n, out, blockSize);
    }
    else if (type == 2) // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }else {
        printf("\nRadix Sort with thrust\n");
        sortWithThrust(in, n, out);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out_baseline = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * out_thrust = (uint32_t *)malloc(bytes);
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);

    // SORT BY BASELINE
    sort(in, n, out_baseline, true, blockSize, 1);
    checkCorrectness(out_baseline, correctOut, n);
    
    // SORT BY DEVICE
    // Calcute avg
    GpuTimer timer; 
    
    float avg_time = 0;
    int loop = 16;
    printf("\nRadix sort by device avg\n");
    for (int i = 0; i < loop; i++){
        timer.Start();
        sort(in, n, out, true, blockSize, 2);
        timer.Stop();
        avg_time += timer.Elapsed();
    }
    printf("AVG TIME: %.f ms\n", avg_time / loop);
    checkCorrectness(out, correctOut, n);

    // SORT BY THRUST
    sort(in, n, out_thrust, true, blockSize, 3);
    checkCorrectness(out_thrust, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1153.050 ms

Baseline Radix Sort (highlight)
Time: 6397.407 ms
CORRECT :)

Radix sort by device avg

Radix Sort by device
Time: 170.193 ms

Radix Sort by device
Time: 141.318 ms

Radix Sort by device
Time: 136.777 ms

Radix Sort by device
Time: 136.682 ms

Radix Sort by device
Time: 135.967 ms

Radix Sort by device
Time: 138.079 ms

Radix Sort by device
Time: 137.696 ms

Radix Sort by device
Time: 137.656 ms

Radix Sort by device
Time: 137.782 ms

Radix Sort by device
Time: 136.865 ms

Radix Sort by device
Time: 136.937 ms

Radix Sort by device
Time: 137.780 ms

Radix Sort by device
Time: 137.190 ms

Radix Sort by device
Time: 136.427 ms

Radix Sort by device
Time: 139.618 ms

Radix Sort by dev

## Radix sort final 2

In [172]:
%%cu
/*
Radix sort final
*/
#include <stdio.h>
#include <stdint.h>
#include <thrust/device_vector.h>
#include <thrust/copy.h>
#include <thrust/sort.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
        cudaEventSynchronize(start);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

// Sequential Radix Sort
void sortByHost(const uint32_t * in, int n,
                uint32_t * out)
{

    int nBits = 4; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    int * hist = (int *)malloc(nBins * sizeof(int));
    int * histScan = (int *)malloc(nBins * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    // Loop from LSD (Least Significant Digit) to MSD (Most Significant Digit)
    // (Each digit consists of nBits bit)
    // In each loop, sort elements according to the current digit from src to dst 
    // (using STABLE counting sort)
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // TODO: Compute histogram
        memset(hist, 0, nBins * sizeof(int));
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            hist[bin]++;
        }

        // TODO: Scan histogram (exclusively)
        histScan[0] = 0;
        for (int bin = 1; bin < nBins; bin++)
            histScan[bin] = histScan[bin - 1] + hist[bin - 1];

        // TODO: Scatter elements to correct locations
        for (int i = 0; i < n; i++)
        {
            int bin = (src[i] >> bit) & (nBins - 1);
            dst[histScan[bin]] = src[i];
            histScan[bin]++;
        }
        
        // Swap src and dst
        uint32_t * temp = src;
        src = dst;
        dst = temp;
    }

    // Copy result to out
   memcpy(out, src, n * sizeof(uint32_t)); 
}

// #########################################################
// Baseline
void sortBaseline(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 1; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize); // block size
    dim3 gridSize((n - 1) / blockSize.x + 1); // grid size

    int * hist = (int *)malloc(nBins * gridSize.x * sizeof(int));
    int *histScan = (int * )malloc(nBins * gridSize.x * sizeof(int));

    uint32_t * src = (uint32_t *)malloc(n * sizeof(uint32_t));
    memcpy(src, in, n * sizeof(uint32_t));
    uint32_t * dst = out;

    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        memset(hist, 0, nBins * gridSize.x * sizeof(int));
        // compute historgram
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            if (i * blockSize.x + j < n)
            {
                int bin = (src[i * blockSize.x + j] >> bit) & (nBins - 1);
                hist[i * nBins + bin]++;
            }
        }

        // compute scan
        int previous = 0;
        for (int j = 0; j < nBins; j++){
            for (int i = 0; i < gridSize.x; i++)
            {
                histScan[i * nBins + j] = previous;
                previous = previous + hist[i * nBins + j];
            }
        }

        // scatter
        for (int i = 0; i < gridSize.x; i++)
        {
            for (int j = 0; j < blockSize.x; j++)
            {
                int id = i * blockSize.x + j;
                if (id < n)
                {
                    int bin = i * nBins + ((src[id] >> bit) & (nBins - 1));
                    dst[histScan[bin]] = src[id];
                    histScan[bin]++;
                }
            }
        }
        uint32_t * temp = src;
        src = dst;
        dst = temp; 
    }

    memcpy(out, src, n * sizeof(uint32_t));
    free(hist);
    free(histScan);
}

// #########################################################
// Radix sort by device
// #########################################################
// Histogram kernel
__global__ void computeHistogram(uint32_t * in, int n, int * hist, int nBins, int bit)
{
    // TODO
    extern __shared__ int s_bin[];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int delta = (nBins - 1) / blockDim.x + 1; // nBins / blockDim

    for (int j = 0; j < delta; j++)
    {
        int id = threadIdx.x + j * blockDim.x;
        if (id < nBins){
            s_bin[id] = 0;
        }
    }
    __syncthreads();

    if (i < n)
    {
        int bin = (in[i] >> bit) & (nBins - 1);
        atomicAdd(&s_bin[bin], 1);
    }
    __syncthreads();

    for (int j = 0; j < delta; j++)
    {
        int id = threadIdx.x + j * blockDim.x;
        if (id < nBins){
            hist[id * gridDim.x + blockIdx.x] += s_bin[id];
        }
    }
}

// scan kernel
__global__ void scanExclusiveBlk(int * in, int n, int * out, int * blkSums)
{   
    // TODO
    extern __shared__ int s_data[];

    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i > 0 && i < n){
        s_data[blockDim.x - 1 - threadIdx.x] = in[i - 1];
    }
    else{
        s_data[blockDim.x - 1 - threadIdx.x] = 0;
    }
    __syncthreads();

    for (int stride = 1; stride < blockDim.x; stride *= 2)
    {
        int val = 0;
        if (threadIdx.x < blockDim.x - stride){
            val = s_data[threadIdx.x + stride];
        }
        __syncthreads();

        s_data[threadIdx.x] += val;
        __syncthreads();
    }

    if (i < n){
        out[i] = s_data[blockDim.x - 1 - threadIdx.x];
    }
    if (blkSums != NULL){
        blkSums[blockIdx.x] = s_data[0];
    }
}

__global__ void computeHistScan(int * in, int n, int* blkSums)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n && blockIdx.x > 0){
        in[i] += blkSums[blockIdx.x - 1];
    }
}

__global__ void scatter(uint32_t * in, int n, int nBits, int bit, int nBins, int *histScan, uint32_t * out)
{
    extern __shared__ int s_data[];
    int * s_in = s_data;
    int * s_hist = (int *)&s_in[blockDim.x];
    int * dst = (int *)&s_hist[blockDim.x];
    int * dst_ori = (int *)&dst[blockDim.x];
    int * startIndex = (int *)&dst_ori[blockDim.x]; // Cấp phát nBins
    int * scan = (int *)&startIndex[nBins]; 
    int * hist = (int *)&scan[blockDim.x];

    int id = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (id < n)
    {
        s_in[threadIdx.x] = in[id];
        s_hist[threadIdx.x] = (s_in[threadIdx.x] >> bit) & (nBins - 1);
    }
    else{
        s_hist[threadIdx.x] = nBins - 1;
    }
    scan[threadIdx.x] = 0;
    __syncthreads();

    // Step 1 : sort radix with k = 1
    for (int b = 0; b < nBits; b++)
    {
        // compute hist
        int _hist = s_hist[threadIdx.x];
        int _in = s_in[threadIdx.x];
        int _bin = (_hist >> b) & 1;
        hist[threadIdx.x] = _bin;
        if (threadIdx.x < blockDim.x - 1){
            scan[threadIdx.x + 1] = _bin;
        }
        __syncthreads();

        int _last_hist = hist[blockDim.x - 1];
        for (int stride = 1; stride < blockDim.x; stride *= 2)
        {
            int val = 0;
            if (threadIdx.x >= stride){
                val = scan[threadIdx.x - stride];
            }
            __syncthreads();

            scan[threadIdx.x] += val;
            __syncthreads();
        }
        __syncthreads();

        // scatter
        int scan_ = scan[threadIdx.x];
        int nZeros = blockDim.x - scan[blockDim.x - 1] - _last_hist;//hist[blockDim.x - 1];
        int rank = 0;
        if (_bin == 0){
            rank = threadIdx.x - scan_;//scan[threadIdx.x];
        }
        else{
            rank = nZeros + scan_;//scan[threadIdx.x];
        }
        dst[rank] = _hist;//s_hist[threadIdx.x];
        dst_ori[rank] = _in;//s_in[threadIdx.x];
        __syncthreads();        
        // copy or swap
        s_hist[threadIdx.x] = dst[threadIdx.x];
        s_in[threadIdx.x] = dst_ori[threadIdx.x];
    }
    int _hist = s_hist[threadIdx.x];
    int _in = s_in[threadIdx.x];
    __syncthreads();

    // Step 2 + 3
    if (threadIdx.x == 0){
        startIndex[_hist] = 0;
    }
    else
    {
        if (_hist != s_hist[threadIdx.x - 1]){
            startIndex[_hist] = threadIdx.x;
        }
    }
    __syncthreads();

    // Step 4 real scatter
    if (id < n)
    {
        int preRank = threadIdx.x - startIndex[_hist];
        int bin = ((_in >> bit) & (nBins - 1));
        int scan = histScan[bin * gridDim.x + blockIdx.x];
        out[scan + preRank] = _in;
    }
}

// Parallel Radix Sort
void sortByDevice(const uint32_t * in, int n, uint32_t * out, int blkSize)
{
    // TODO
    int nBits = 8; // Assume: nBits in {1, 2, 4, 8, 16}
    int nBins = 1 << nBits; // 2^nBits

    dim3 blockSize(blkSize);
    dim3 gridHistSize((n - 1) / blockSize.x + 1);
    dim3 gridScanSize((nBins * gridHistSize.x - 1) / blockSize.x + 1);
    dim3 gridScatterSize((n - 1) / blockSize.x + 1);
    
    int * blkSums = (int *)malloc(gridScanSize.x * sizeof(int));
    uint32_t * d_src, *d_dst;
    CHECK(cudaMalloc(&d_src, n * sizeof(uint32_t)));
    CHECK(cudaMemcpy(d_src, in, n * sizeof(uint32_t), cudaMemcpyHostToDevice)); // copy to device
    CHECK(cudaMalloc(&d_dst, n * sizeof(uint32_t)));

    int *d_scan, *d_blkSums;
    CHECK(cudaMalloc(&d_scan, nBins * gridHistSize.x * sizeof(int)));
    CHECK(cudaMalloc(&d_blkSums, gridScanSize.x * sizeof(int)));

    size_t smemHistBytes = nBins * sizeof(int);
    size_t smemScanBytes = blockSize.x * sizeof(int);
    size_t smemScatterBytes = (blockSize.x * 6 +  nBins) * sizeof(int);
    
    for (int bit = 0; bit < sizeof(uint32_t) * 8; bit += nBits)
    {
        // Compute "hist" of the current digit
        CHECK(cudaMemset(d_scan, 0, nBins * gridHistSize.x * sizeof(int)));
        computeHistogram<<<gridHistSize, blockSize, smemHistBytes>>>(d_src, n, d_scan, nBins, bit);
        cudaDeviceSynchronize();
        
        // Scan
        scanExclusiveBlk<<<gridScanSize, blockSize, smemScanBytes>>>(d_scan, nBins * gridHistSize.x, d_scan, d_blkSums);
        cudaDeviceSynchronize();
        CHECK(cudaMemcpy(blkSums, d_blkSums, gridScanSize.x * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 1; i < gridScanSize.x; i++){
            blkSums[i] += blkSums[i - 1];
        }
        CHECK(cudaMemcpy(d_blkSums, blkSums, gridScanSize.x * sizeof(int), cudaMemcpyHostToDevice));
        computeHistScan<<<gridScanSize, blockSize>>>(d_scan, nBins * gridHistSize.x, d_blkSums);
        cudaDeviceSynchronize();

        // Scatter
        scatter<<<gridScatterSize, blockSize, smemScatterBytes>>>(d_src, n, nBits, bit, nBins, d_scan, d_dst);
        cudaDeviceSynchronize();
        
        // Swap "src" and "dst"
        uint32_t * temp = d_src;
        d_src = d_dst;
        d_dst = temp;
    }
    // Copy "d_src" to "out"
    CHECK(cudaMemcpy(out, d_src, n * sizeof(uint32_t), cudaMemcpyDeviceToHost));

    CHECK(cudaFree(d_src));
    CHECK(cudaFree(d_dst));
    CHECK(cudaFree(d_scan));
    CHECK(cudaFree(d_blkSums));
    
    free(blkSums);
}

// #########################################################
// Radix sort by thrust
// #########################################################
void sortWithThrust(const uint32_t * in, int n, uint32_t * out)
{
    thrust::device_vector<uint32_t> dv_out(in, in + n);
    thrust::sort(dv_out.begin(), dv_out.end());
    thrust::copy(dv_out.begin(), dv_out.end(), out);
}

// Radix Sort
void sort(const uint32_t * in, int n, 
        uint32_t * out, 
        bool useDevice=false, int blockSize=1, int type=0)
{
    GpuTimer timer; 
    timer.Start();

    if (useDevice == false)
    {
        printf("\nRadix Sort by host\n");
        sortByHost(in, n, out);
    }
    else if (type == 1){ // Baseline
        printf("\nBaseline Radix Sort (highlight)\n");
        sortBaseline(in, n, out, blockSize);
    }
    else if (type == 2) // use device
    {
        printf("\nRadix Sort by device\n");
        sortByDevice(in, n, out, blockSize);
    }else {
        printf("\nRadix Sort with thrust\n");
        sortWithThrust(in, n, out);
    }

    timer.Stop();
    printf("Time: %.3f ms\n", timer.Elapsed());
}

void printDeviceInfo()
{
    cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %zu byte\n", devProv.totalGlobalMem);
    printf("SMEM per SM: %zu byte\n", devProv.sharedMemPerMultiprocessor);
    printf("SMEM per block: %zu byte\n", devProv.sharedMemPerBlock);
    printf("****************************\n");
}

void checkCorrectness(uint32_t * out, uint32_t * correctOut, int n)
{
    for (int i = 0; i < n; i++)
    {
        if (out[i] != correctOut[i])
        {
            printf("INCORRECT :(\n");
            return;
        }
    }
    printf("CORRECT :)\n");
}

void printArray(uint32_t * a, int n)
{
    for (int i = 0; i < n; i++)
        printf("%i ", a[i]);
    printf("\n");
}

int main(int argc, char ** argv)
{
    // PRINT OUT DEVICE INFO
    printDeviceInfo();

    // SET UP INPUT SIZE
    int n = (1 << 24) + 1; // For test by eye
    //int n = (1 << 24) + 1;
    printf("\nInput size: %d\n", n);

    // ALLOCATE MEMORIES
    size_t bytes = n * sizeof(uint32_t);
    uint32_t * in = (uint32_t *)malloc(bytes);
    uint32_t * out_baseline = (uint32_t *)malloc(bytes);
    uint32_t * out = (uint32_t *)malloc(bytes); // Device result
    uint32_t * out_thrust = (uint32_t *)malloc(bytes);
    uint32_t * correctOut = (uint32_t *)malloc(bytes); // Host result

    // SET UP INPUT DATA
    for (int i = 0; i < n; i++)
    {
        in[i] = rand() % 255; // For test by eye
        //in[i] = rand();
    }
    // printArray(in, n); // For test by eye

    // DETERMINE BLOCK SIZE
    int blockSize = 512; // Default
    if (argc == 2)
        blockSize = atoi(argv[1]);

    // SORT BY HOST
    sort(in, n, correctOut);
    // printArray(correctOut, n);

    // SORT BY BASELINE
    sort(in, n, out_baseline, true, blockSize, 1);
    checkCorrectness(out_baseline, correctOut, n);
    
    // SORT BY DEVICE
    sort(in, n, out, true, blockSize, 2);
    checkCorrectness(out, correctOut, n);    

    // SORT BY THRUST
    sort(in, n, out_thrust, true, blockSize, 3);
    checkCorrectness(out_thrust, correctOut, n);

    // FREE MEMORIES
    free(in);
    free(out);
    free(correctOut);
    
    return EXIT_SUCCESS;
}


**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 1136.623 ms

Baseline Radix Sort (highlight)
Time: 6289.531 ms
CORRECT :)

Radix Sort by device
Time: 115.452 ms
CORRECT :)

Radix Sort with thrust
Time: 65.580 ms
CORRECT :)



# C. Build and run with different block size

In [173]:
!nvcc build.cu -o build

### blockSize = 128

In [174]:
!./build 128

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 128

Radix Sort by host
Time: 1155.428 ms

Baseline Radix Sort (highlight)
Time: 6378.104 ms
CORRECT :)

Radix Sort by device
Time: 144.930 ms
CORRECT :)

Radix Sort with thrust
Time: 64.635 ms
CORRECT :)


### blockSize = 256

In [175]:
!./build 256

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 256

Radix Sort by host
Time: 1142.361 ms

Baseline Radix Sort (highlight)
Time: 6392.780 ms
CORRECT :)

Radix Sort by device
Time: 126.807 ms
CORRECT :)

Radix Sort with thrust
Time: 64.410 ms
CORRECT :)


### blockSize = 512

In [176]:
!./build 512

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 512

Radix Sort by host
Time: 1157.686 ms

Baseline Radix Sort (highlight)
Time: 6223.896 ms
CORRECT :)

Radix Sort by device
Time: 107.973 ms
CORRECT :)

Radix Sort with thrust
Time: 61.450 ms
CORRECT :)


### blockSize = 1024

In [185]:
!./build 1024

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 1024

Radix Sort by host
Time: 1163.608 ms

Baseline Radix Sort (highlight)
Time: 6285.182 ms
CORRECT :)

Radix Sort by device
Time: 120.331 ms
CORRECT :)

Radix Sort with thrust
Time: 66.337 ms
CORRECT :)


# D. Profiling radix program

In [4]:
!nvcc build.cu -o build -Wno-deprecated-gpu-targets

In [5]:
!nvprof ./build

==230== NVPROF is profiling process 230, command: ./build
**********GPU info**********
Name: Tesla V100-SXM2-16GB
Compute capability: 7.0
Num SMs: 80
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 16914055168 byte
SMEM per SM: 98304 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 512

Radix Sort by host
Time: 1246.156 ms

Baseline Radix Sort (highlight)
Time: 6712.014 ms
CORRECT :)

Radix Sort by device
Time: 79.376 ms
CORRECT :)

Radix Sort with thrust
Time: 63.225 ms
CORRECT :)
==230== Profiling application: ./build
==230== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.51%  87.110ms         6  14.518ms  6.2720us  43.572ms  [CUDA memcpy DtoH]
                   22.25%  29.592ms         6  4.9320ms  8.5110us  14.789ms  [CUDA memcpy HtoD]
                    7.77%  10.335ms         4  2.5838ms  2.5509ms  2.6823ms  scatter(unsigned int*, int, int, int, 

In [6]:
!time ./build

**********GPU info**********
Name: Tesla V100-SXM2-16GB
Compute capability: 7.0
Num SMs: 80
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 16914055168 byte
SMEM per SM: 98304 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217
Block size : 512

Radix Sort by host
Time: 1216.432 ms

Baseline Radix Sort (highlight)
Time: 6724.532 ms
CORRECT :)

Radix Sort by device
Time: 79.986 ms
CORRECT :)

Radix Sort with thrust
Time: 63.022 ms
CORRECT :)

real	0m8.775s
user	0m8.408s
sys	0m0.342s
